# Implementation of "adversarial training" as a defence mechanism

## 1. Training on augmented dataset
### 1.1 Augmenting the dataset

### 1.2 Training

### 1.3 Benchmark
We compare the performance of two networks (classical and augmented training) on adversarial examples.

## 2. Training with modified loss function
We follow the approach described by Goodfellow et al. and change the loss function to:
$$\tilde{J}_\theta(x, y) = \alpha J_\theta(x, y) + (1-\alpha)J_\theta\left(x+\varepsilon\text{ sign}\left(\nabla_x J_\theta(x, y)\right)\right)$$

### 2.1 Training

### 2.2 Benchmark